In [1]:
using Laplacians

objc[55046]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[55046]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[55046]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[55046]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/se_5/"

"../graphs/se_5/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
x = readFromFile(graphp * "x.vec");

In [8]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [11]:
@time F = treeSolver(tree)
@time F(b);

  0.454831 seconds (1.25 M allocations: 86.056 MB, 3.43% gc time)
  0.076113 seconds (2.00 M allocations: 38.109 MB, 14.31% gc time)


In [15]:
# a run on standard Float64 data types
numIts = 150;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 307443.3614437539
Working on iteration 20 with a-norm error 8001.255087898712
Working on iteration 30 with a-norm error 1057.4237137141467
Working on iteration 40 with a-norm error 134.1108676622286
Working on iteration 50 with a-norm error 40.21016416959214
Working on iteration 60 with a-norm error 2.7144290892194904
Working on iteration 70 with a-norm error 0.6485248355624706
Working on iteration 80 with a-norm error 0.05461200810220714
Working on iteration 90 with a-norm error 0.01311180760784604
Working on iteration 100 with a-norm error 0.003541115599785892
Working on iteration 110 with a-norm error 0.0005683546325843358
Working on iteration 120 with a-norm error 6.555585239552845e-5
Working on iteration 130 with a-norm error 1.6343191775630755e-5
Working on iteration 140 with a-norm error 1.4157523393277654e-6
Working on iteration 150 with a-norm error 8.648800094156629e-7
PCG stopped after: 150 iterations with relative error 8.6472984559

In [16]:
numIts = 150
for hp in [64,128,512,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 307443.36146129033
Working on iteration 20 with a-norm error 7892.203201233427
Working on iteration 30 with a-norm error 1057.4237133597633
Working on iteration 40 with a-norm error 132.9162483822737
Working on iteration 50 with a-norm error 32.04401646395055
Working on iteration 60 with a-norm error 2.7077362028973293
Working on iteration 70 with a-norm error 0.17856095693367358
Working on iteration 80 with a-norm error 0.04804644323563823
Working on iteration 90 with a-norm error 0.004762776357510621
Working on iteration 100 with a-norm error 0.0009613881731549992
Working on iteration 110 with a-norm error 0.00022918566099241138
Working on iteration 120 with a-norm error 1.6936193562291736e-5
Working on iteration 130 with a-norm error 1.2194410779165358e-6
Working on iteration 140 with a-norm error 3.0105427726434275e-7
Working on iteration 150 with a-norm error 7.796464241945001e-8
PCG stopped after: 150 iterations with relative error 7.5560